# ML model to recognize two-phase flows regimes

## Import modules

In [1]:
# Import modules
# ------------------------------------------------------------------------------ #
# Built-in modules
import os
import glob
import random

# Third-party modules
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
# ------------------------------------------------------------------------------ #

## Image data generator

In [2]:
# Input data
# ------------------------------------------------------------------------------ #
n_img = 10 # number of images to take from each run

# Labels
# 1 - Plug Flow; 2 - Transition Flow; 3 - Slug Flow

# Initialize lists
training_imgs = []
training_labels = []

# ------------------------------------------------------------------------------ #
# Data from Slug-Frequency campaign
# ------------------------------------------------------------------------------ #
IJMF_dir = '/media/psassi/URV/Paolo/URV/data/FastCam/3_Freq/Velocity/2P'
IJMF_regimes = [2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
#
dir_contents = os.listdir(IJMF_dir)
dir_contents.sort()
two_phase_runs = []
for run in dir_contents:
    if '2P' in run:
        two_phase_runs.append(os.path.join(IJMF_dir,run))
two_phase_runs.sort()
#
for i, run in enumerate(two_phase_runs):
    imgs_path = os.path.join(run, 'PPS_images', "*png")
    files = glob.glob(imgs_path)
    training_imgs.extend(random.sample(files,n_img))
    training_labels.extend([IJMF_regimes[i]]*n_img)
# ------------------------------------------------------------------------------ #

In [5]:
# ------------------------------------------------------------------------------ #
# Data from ECMF campaign
# ------------------------------------------------------------------------------ #
FTaC_dir = '/media/psassi/URV/Paolo/URV/data/FastCam/2_FTaC/Backlight/2P'
FTaC_regimes = [1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 3, 3, 2, 3, 3]
#
dir_contents = os.listdir(FTaC_dir)
dir_contents.sort()
FTaC_runs = []
for run in dir_contents:
    if '2P' in run:
        FTaC_runs.append(os.path.join(FTaC_dir,run))
FTaC_runs.sort()
#
for i, run in enumerate(FTaC_runs):
    imgs_path = os.path.join(run, 'PNG_images', "*png")
    files = glob.glob(imgs_path)
    training_imgs.extend(random.sample(files,n_img))
    training_labels.extend([FTaC_regimes[i]]*n_img)
# ------------------------------------------------------------------------------ #

In [15]:
# Make pandas Data Frame
two_phase_pd = pd.DataFrame(
    list(zip(training_imgs,training_labels)), columns=['Files', 'Labels']
    )

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator().flow_from_dataframe(
    two_phase_pd,
    x_col='Files',
    y_col='Labels',
    target_size=(512, 256),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=32,
)

train_generator